# T4D

In [1]:
from datasets import Dataset
from pyprojroot import here

In [2]:
chk_dir = here("struct_vs_unstruct/data/mistral_large_2407/original/non_self_synthesis/t4d/t4d-/t4d_eval")

In [3]:
dataset = Dataset.load_from_disk(chk_dir)

In [11]:
dataset

Dataset({
    features: ['story', 'question', 'answer', 'characters', 'distracting_characters', 'reasoning_formats', 'selected_modules', 'adapted_modules', 'reasoning_structure', 'reasoning', 'trajectory', 'answer_pred'],
    num_rows: 564
})

In [5]:
set(dataset["answer_pred"])

{'A. Abigail and C. Jacob.',
 'A. Abigail"',
 'A. Abigail.',
 'A. Aiden and C. Mason.',
 'A. Aiden.',
 'A. Alexander and B. Jackson."',
 'A. Alexander"',
 'A. Alexander.',
 'A. Amelia and B. Sophia.',
 'A. Amelia and C. Ella."',
 'A. Amelia"',
 'A. Amelia.',
 'A. Amelia."',
 'A. Aria.',
 'A. Ava and B. Ethan.',
 'A. Ava"',
 'A. Ava.',
 'A. Avery"',
 'A. Avery.',
 'A. Benjamin"',
 'A. Benjamin.',
 'A. Carter.',
 'A. Charlotte"',
 'A. Charlotte.',
 'A. Chloe.',
 'A. Chloe."',
 'A. Elizabeth.',
 'A. Elizabeth."',
 'A. Ella.',
 'A. Emily.',
 'A. Emily."',
 'A. Emma and B. Lily."',
 'A. Emma"',
 'A. Emma.',
 'A. Emma."',
 'A. Ethan and C. William.',
 'A. Ethan"',
 'A. Ethan.',
 'A. Ethan."',
 'A. Evelyn.',
 'A. Hannah"',
 'A. Hannah.',
 'A. Hunter.',
 'A. Hunter."',
 'A. Hunter.**',
 'A. Isabella"',
 'A. Isabella.',
 'A. Isla.',
 'A. Isla."',
 'A. Jack.',
 'A. Jack."',
 'A. Jackson and C. Ava."',
 'A. Jackson.',
 'A. Jackson."',
 'A. Jacob"',
 'A. Jacob.',
 'A. Jacob."',
 'A. James"',
 'A. 

In [14]:
def map_fn(instance):    
    return {
        "answer_pred": instance["answer_pred"].translate(str.maketrans("", "", '.*"')) if instance["answer_pred"] else None
    }

new_ds = dataset.map(map_fn, load_from_cache_file=False)

Map:   0%|          | 0/564 [00:00<?, ? examples/s]

In [15]:
set(new_ds["answer_pred"])

{'A Abigail',
 'A Abigail and C Jacob',
 'A Aiden',
 'A Aiden and C Mason',
 'A Alexander',
 'A Alexander and B Jackson',
 'A Amelia',
 'A Amelia and B Sophia',
 'A Amelia and C Ella',
 'A Aria',
 'A Ava',
 'A Ava and B Ethan',
 'A Avery',
 'A Benjamin',
 'A Carter',
 'A Charlotte',
 'A Chloe',
 'A Elizabeth',
 'A Ella',
 'A Emily',
 'A Emma',
 'A Emma and B Lily',
 'A Ethan',
 'A Ethan and C William',
 'A Evelyn',
 'A Hannah',
 'A Hunter',
 'A Isabella',
 'A Isla',
 'A Jack',
 'A Jackson',
 'A Jackson and C Ava',
 'A Jacob',
 'A James',
 'A Jayden',
 'A Liam',
 'A Liam and B Aiden',
 'A Lily',
 'A Lily and B Liam',
 'A Logan',
 'A Lucas',
 "A Lucas Lucas needs to know the radish's current location to plan its use effectively, as Jayden moved it to the crate without Lucas's knowledge",
 'A Mason',
 'A Mia',
 'A Mia and B Jayden',
 'A Mila',
 'A Nathan',
 'A Noah',
 'A Oliver',
 'A Olivia',
 'A Owen',
 'A Sophia',
 'A Sophia and C Logan',
 'A William',
 'B Abigail',
 'B Aiden',
 'B Aide

In [17]:
def map_fn(ins):
    if "A Lucas Lucas needs to know" in ins["answer_pred"]:
        return {
            "answer_pred": "A Lucas"
        }

    return {
        "answer_pred": ins["answer_pred"]
    }

new_ds = new_ds.map(map_fn)
set(new_ds["answer_pred"])

Map:   0%|          | 0/564 [00:00<?, ? examples/s]

{'A Abigail',
 'A Abigail and C Jacob',
 'A Aiden',
 'A Aiden and C Mason',
 'A Alexander',
 'A Alexander and B Jackson',
 'A Amelia',
 'A Amelia and B Sophia',
 'A Amelia and C Ella',
 'A Aria',
 'A Ava',
 'A Ava and B Ethan',
 'A Avery',
 'A Benjamin',
 'A Carter',
 'A Charlotte',
 'A Chloe',
 'A Elizabeth',
 'A Ella',
 'A Emily',
 'A Emma',
 'A Emma and B Lily',
 'A Ethan',
 'A Ethan and C William',
 'A Evelyn',
 'A Hannah',
 'A Hunter',
 'A Isabella',
 'A Isla',
 'A Jack',
 'A Jackson',
 'A Jackson and C Ava',
 'A Jacob',
 'A James',
 'A Jayden',
 'A Liam',
 'A Liam and B Aiden',
 'A Lily',
 'A Lily and B Liam',
 'A Logan',
 'A Lucas',
 'A Mason',
 'A Mia',
 'A Mia and B Jayden',
 'A Mila',
 'A Nathan',
 'A Noah',
 'A Oliver',
 'A Olivia',
 'A Owen',
 'A Sophia',
 'A Sophia and C Logan',
 'A William',
 'B Abigail',
 'B Aiden',
 'B Aiden and C Hunter',
 'B Alexander',
 'B Amelia',
 'B Aria',
 'B Aria and C Olivia',
 'B Ava',
 'B Ava and C Avery',
 'B Avery',
 'B Benjamin',
 'B Carte

In [18]:
t4d = lambda instance, y, y_pred: instance[y] in instance[y_pred] and instance[y] == str(instance[y_pred].translate(str.maketrans("", "", "."))[2:])

corr = new_ds.filter(lambda x: t4d(x, "answer", "answer_pred"))
corr.num_rows / new_ds.num_rows

Filter:   0%|          | 0/564 [00:00<?, ? examples/s]

0.7606382978723404

In [19]:
new_ds.push_to_hub("sachithgunasekara/self-discover-mistral-original-t4d-eval")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

CommitInfo(commit_url='https://huggingface.co/datasets/sachithgunasekara/self-discover-mistral-original-t4d-eval/commit/b8d386d021020aa571ae3b32b5b22371e6753664', commit_message='Upload dataset', commit_description='', oid='b8d386d021020aa571ae3b32b5b22371e6753664', pr_url=None, pr_revision=None, pr_num=None)